In [2]:
from fastai import *
from fastai.tabular import *

In [7]:
tabular_learner??

In [ ]:
#Signature: tabular_learner(data:fastai.basic_data.DataBunch, layers:Collection[int], emb_szs:Dict[str, int]=None, metrics=None, ps:Collection[float]=None, emb_drop:float=0.0, y_range:Union[Tuple[float, float], NoneType]=None, use_bn:bool=True, **learn_kwargs)
def tabular_learner(data:DataBunch, layers:Collection[int], emb_szs:Dict[str,int]=None, metrics=None,
        ps:Collection[float]=None, emb_drop:float=0., y_range:OptRange=None, use_bn:bool=True, **learn_kwargs):
    "Get a `Learner` using `data`, with `metrics`, including a `TabularModel` created using the remaining params."
    emb_szs = data.get_emb_szs(ifnone(emb_szs, {}))
    model = TabularModel(emb_szs, len(data.cont_names), out_sz=data.c, layers=layers, ps=ps, emb_drop=emb_drop,
                         y_range=y_range, use_bn=use_bn)
    return Learner(data, model, metrics=metrics, **learn_kwargs)

In [3]:
tabular_learner??

In [55]:
nn.Dropout??

In [3]:
class TabularModel(Module):
    "Basic model for tabular data."
    def __init__(self, emb_szs:ListSizes, n_cont:int, out_sz:int, layers:Collection[int], ps:Collection[float]=None,
                 emb_drop:float=0., y_range:OptRange=None, use_bn:bool=True, bn_final:bool=False):
        super().__init__()
        ps = ifnone(ps, [0]*len(layers))
        ps = listify(ps, layers)
        self.embeds = nn.ModuleList([embedding(ni, nf) for ni,nf in emb_szs])
        self.emb_drop = nn.Dropout(emb_drop)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        n_emb = sum(e.embedding_dim for e in self.embeds)
        self.n_emb,self.n_cont,self.y_range = n_emb,n_cont,y_range
        sizes = self.get_sizes(layers, out_sz)
        actns = [nn.ReLU(inplace=True) for _ in range(len(sizes)-2)] + [None]
        layers = []
        for i,(n_in,n_out,dp,act) in enumerate(zip(sizes[:-1],sizes[1:],[0.]+ps,actns)):
            layers += bn_drop_lin(n_in, n_out, bn=use_bn and i!=0, p=dp, actn=act)
        if bn_final: layers.append(nn.BatchNorm1d(sizes[-1]))
        self.layers = nn.Sequential(*layers)

    def get_sizes(self, layers, out_sz):
        return [self.n_emb + self.n_cont] + layers + [out_sz]

    def forward(self, x_cat:Tensor, x_cont:Tensor) -> Tensor:
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]
            x = torch.cat(x, 1)
            x = self.emb_drop(x)
        if self.n_cont != 0:
            x_cont = self.bn_cont(x_cont)
            x = torch.cat([x, x_cont], 1) if self.n_emb != 0 else x_cont
        x = self.layers(x)
        if self.y_range is not None:
            x = (self.y_range[1]-self.y_range[0]) * torch.sigmoid(x) + self.y_range[0]
        return x

## Match Pred Model

How will this model work?

Well, in simple terms we will have shared weights for A and B inpus where A and B are the exact same information but for the 2 different contestants, this information can be as big as desired. Then we will bind in the general stats such as weather, time and related stuff.

In [ ]:
class MatchTabularModel(nn.Module):
    "Basic model for match tabular data."
    def __init__(self,
                 #Contestants parameters
                 emb_szs_cts:ListSizes, n_cont_cts:int, layers_cts:Collection[int], ps_cts:Collection[float]=None, emb_drop_cts:float=0.,
                 #General Data parameters
                 emb_szs_grl:ListSizes, n_cont_grl:int, layers_grl:Collection[int], ps_grl:Collection[float]=None, emb_drop_grl:float=0.,
                 #General Model parameters
                 use_bn:bool=True, bn_final:bool=False, out_sz:int, y_range:OptRange=None):
        super().__init__()
        
        """Contestants Setup"""
        # Dropout probabilities
        ps_cts = ifnone(ps_cts, [0]*len(layers_cts))
        ps_cts = listify(ps_cts, layers_cts)
        
        # Embeddings
        self.embeds_cts = nn.ModuleList([embedding(ni, nf) for ni,nf in emb_szs_cts])
        self.emb_drop_cts = nn.Dropout(emb_drop_cts)
        n_emb_cts = sum(e.embedding_dim for e in self.embeds_cts)
        
        # Continious (non-embedding)
        self.bn_cont_cts = nn.BatchNorm1d(n_cont_cts)
        
        # Embeddings and Continious
        self.n_emb_cts, self.n_cont_cts = n_emb_cts, n_cont_cts
        sizes_cts = self.get_sizes(n_emb_cts, n_cont_cts, layers_cts)
        actns_cts = [nn.ReLU(inplace=True) for _ in range(len(sizes_cts)-1)]
        layers_cts = []
        for i,(n_in, n_out, dp, act) in enumerate(zip(sizes_cts[:-1], sizes_cts[1:], [0.]+ps_cts, actns_cts)):
            layers_cts += bn_drop_lin(n_in, n_out, bn=use_bn and i!=0, p=dp, actn=act)
        self.layers_cts = layers_cts    
        
        """General Setup"""
        # Dropout Probabilities
        ps_grl = ifnone(ps_grl, [0]*len(layers_grl))
        ps_grl = listify(ps_grl, layers_grl)
        
        # Embeddings
        self.embeds_grl = nn.ModuleList([embedding(ni, nf) for ni,nf in emb_szs_grl])
        self.emb_drop_grl = nn.Dropout(emb_drop_grl)
        n_emb_grl = sum(e.embedding_dim for e in self.embeds_grl)
        
        # Continious (non-embedding)
        n_cont_grl += sizes_cts[-1] # adding output from contestants subnet to the size of the continious input
        self.bn_cont_grl = nn.BatchNorm1d(n_cont_grl)
        
        # Embeddings and Continitous
        self.n_emb_grl, self.n_cont_grl = n_emb_grl, n_cont_grl
        sizes_grl = self.get_sizes(n_emb_grl, n_cont_grl, layers_grl, out_sz)
        actns_grl = [nn.ReLU(inplace=True) for _ in range(len(sizes_grl)-2)] + [None]
        layers_grl = []
        for i,(n_in, n_out, dp, act) in enumerate(zip(sizes_grl[:-1], sizes_grl[1:], [0.]+ps_grl, actns_grl)):
            layers_grl += bn_drop_lin(n_in, n_out, bn=use_bn and i!=0, p=dp, actn=act)
        
        if bn_final: layers_grl.append(nn.BatchNorm1d(sizes_grl[-1]))
        self.layers_grl = nn.Sequential(*layers_grl)
    
        """General Model Setup"""
        self.y_range = y_range

    def get_sizes(self, n_emb, n_cont, layers, out_sz = None):
        res = [n_emb + n_cont] + layers
        return  res + [out_sz] if out_sz is not None else res

    def forward(self,
                # Contestant A
                cat_a:Tensor, cont_a:Tensor,
                # Contestant B
                cat_b:Tensor, cont_b:Tensor,
                # Genearal data
                cat_grl:Tensor, cont_grl:Tensor) -> Tensor:
        
        # Forward of Contestant Layers:
        if self.n_emb_cts != 0:
            a = [e(cat_a[:,i]) for i,e in enumerate(self.embeds_cts)]
            b = [e(cat_a[:,i]) for i,e in enumerate(self.embeds_cts)]
            
            a = torch.cat(a, 1)
            b = torch.cat(b, 1)
            
            # Not very sure about this step, check here and in layers.
            # TODO: implement later: https://discuss.pytorch.org/t/how-to-fix-the-dropout-mask-for-different-batch/7119/3
            #a = a.emb_drop_cts(a)
            #b = b.emb_drop_cts(b)
        
        if self.n_cont_cts != 0:
            # not sure about this step, check here and in layers
            # TODO: check best practice for the batchnorm (use the same normalization for both or not)
            cont_a = self.bn_cont_cts(cont_a)
            cont_b = self.bn_cont_cts(cont_b)
            
            a = torch.cat([a, cont_a], 1) if self.n_emb_cts != 0 else cont_a
            b = torch.cat([b, cont_b], 1) if self.n_emb_cts != 0 else cont_b
            
        for lyr in self.layers_cts:
            a = lyr(a)
            b = lyr(b)
        
        # SUM
        x = a+b
        
        if self.n_emb_grl != 0:
            x_cat = [e(cat_grl[:,i]) for i,e in enumerate(self.embeds_grl)]
            x_cat = torch.cat(x_cat, 1)
            x_cat = self.emb_drop_grl(x_cat)
            x = torch.cat([x, x_cat], 1)
        if self.n_cont != 0:
            x_cont = self.bn_cont_grl(cont_grl)
            x = torch.cat([x, x_cont], 1) if self.n_emb_grl != 0 else x_cont
            
        x = self.layers_grl(x)
        if self.y_range is not None:
            x = (self.y_range[1]-self.y_range[0]) * torch.sigmoid(x) + self.y_range[0]
        return x

In [14]:
listify??

In [12]:
a = [1, 2, 3]
b = ['a', 'b', 'c']

for i, (x, y) in enumerate(zip(a[:-1], b)):
    print(i, x, y)

0 1 a
1 2 b


In [14]:
._DropoutNd??

Object `_DropoutNd` not found.


In [15]:
nn._DropoutNd??

Object `nn._DropoutNd` not found.


In [71]:
y = Tensor([[1, 1, 1],
           [1, 1, 1],
           [1, 1, 1]])

drp = nn.Dropout(0.5)

0.0

In [74]:
nn.BatchNorm1d??